## Third party ADP library prevalence analysis
- Find the number sites that embed a given script/endpoint

In [1]:
"""
ADP_BASE_URLS_REGEXES = {
    "Fomo": "^e.fomo.com.*events.js",
    "Beeketing": "^gapi.beeketing.com/.*/get-sdk-notifications",
    "Recently": "^recently.appifiny.io/store",
    "Fera": "^app.fera.ai/api/v1/content/containers.json",
    "Vitals": "^getvitals.io/assets/js/bundle.js",
    "Nice (Shopify plugin)": "^www.goldendev.win/shopify/show-recent-orders/get/last-order.php",
    "Amasty (Magento plugin)": ".*amwhatsup/block/getlastactivity/",
    "LeanConvert": "lc-api.net/v1/product/events",
    "Taggstar": "^cdn.taggstar.com",
    "Insider": "^api.useinsider.com",
    "FreshRelevance": "^dn1i8v75r669j.cloudfront.net",
    "Qubit": "^static.goqubit.com/smartserve",
    "Bunting": "bunting.com/call.js",
    "ConvertCart": "convertcart.com",
    "Proof": "^api.useproof.com",
    "Convertize": "^cp.convertize.io/scarcity",
    "Boost (Wordpress plugin)": "plugins/boost/public/js/boost-public.js",
    "Woocommerce Notification (Woocommerce plugin)": "plugins/woocommerce-notification/js/woocommerce-notification.min.js",
    "Credibly": "^happy-shoppers.credibly.io/shopify/script.js",
    "DynamicYield": "^cdn.dynamicyield.com/api/.*/api_.*.js",
    "Bizzy": "pxu-recent-sales-apps.s3.amazonaws.com/recent-sales/.*/events.js",
    "Exponea": "api.exponea.com/js/exponea.min.js",
    "Yieldify": "yieldify.com"
}
"""

ADP_PROVIDER_DOMAINS = {
    "Fomo": "fomo.com",
    "Beeketing": "beeketing.com",
    "Recently": "appifiny.io",
    "Fera": "fera.ai",
    "Vitals": "getvitals.io",
    "Nice (Shopify plugin)": "goldendev.win",
    "LeanConvert": "lc-api.net",
    
    "Amasty (Magento plugin)": "#amwhatsup/block/getlastactivity",
    "Boost (Wordpress plugin)": "#plugins/boost/public/js/boost",
    "Woocommerce Notification (Woocommerce plugin)": "#plugins/woocommerce-notification",

    "Taggstar": "taggstar.com",
    "Insider": "useinsider.com",
    "FreshRelevance": "dn1i8v75r669j.cloudfront.net",
    "Qubit": "goqubit.com",
    "Bunting": "bunting.com",
    "ConvertCart": "convertcart.com",
    "Proof": "useproof.com",
    "Convertize": "convertize.io",
    
    "Credibly": "credibly.io",
    "DynamicYield": "dynamicyield.com",
    "Bizzy": "pxu-recent-sales-apps.s3.amazonaws.com",
    "Exponea": "exponea.com",
    "Yieldify": "yieldify.com"
}


In [2]:
import re
import json
import sqlite3
import pandas as pd
import json
from collections import defaultdict
from urlparse import urlparse
from os.path import expanduser
from extract_tp_to_site_mapping import get_tld_or_host
pd.set_option("display.max_colwidth",500)
pd.set_option("display.max_rows",500)


In [3]:
def dump_as_json(obj, json_path):
    with open(json_path, 'w') as f:
        json.dump(obj, f)

def load_json_file(json_path):
    with open(json_path) as json_file:
        return json.load(json_file)


In [4]:
ODIN_DB_PATH = "/media/gacar/Data/dp/20190206-203758_segmentation_pilot/20190206-203758_segmentation_pilot.sqlite"
WEBTAP_DB_PATH = "/media/gacar/Data/dp/20190206-205000_segmentation_pilot/20190206-205000_segmentation_pilot.sqlite"

In [5]:
def get_embedding_sites_by_regex(endpoint_regexes, db_path):
    con = sqlite3.connect(db_path)
    con.row_factory = sqlite3.Row
    query = """SELECT sv.visit_id, sv.site_url, r.url, r.method,
                r.post_body FROM http_requests as r LEFT JOIN site_visits as sv
                ON sv.visit_id = r.visit_id
                """
    adp_sites = defaultdict(set)
    # print("Will run the query %s " % query)
    for row in con.execute(query):
        for endpoint_name, endpoint_regex in endpoint_regexes.iteritems():
            if re.search(endpoint_regex, row['url'].split("://")[-1]):
                host = urlparse(row['site_url']).hostname
                # print host, row['site_url']
                adp_sites[endpoint_name].add(host)

    return adp_sites


def get_embedding_sites(url_substrings, db_path):
    con = sqlite3.connect(db_path)
    con.row_factory = sqlite3.Row
    query = """SELECT sv.visit_id, sv.site_url, r.url, r.method,
                r.post_body FROM http_requests as r LEFT JOIN site_visits as sv
                ON sv.visit_id = r.visit_id"""
    adp_sites = defaultdict(set)
    # print("Will run the query %s " % query)
    for row in con.execute(query):
        for js_url in url_substrings:
            if js_url in row['url']:
                host = urlparse(row['site_url']).hostname
                # print host, row['site_url']
                adp_sites[js_url].add(host)

    return adp_sites

def get_embedding_sites_by_domain(endpoint_patterns, db_path):
    con = sqlite3.connect(db_path)
    con.row_factory = sqlite3.Row
    query = """SELECT sv.visit_id, sv.site_url, r.url FROM
                http_requests as r LEFT JOIN site_visits as sv
                ON sv.visit_id = r.visit_id
                """
    adp_sites = defaultdict(set)
    # print("Will run the query %s " % query)
    for row in con.execute(query):
        for endpoint_name, endpoint_pattern in endpoint_patterns.iteritems():
            if endpoint_pattern.startswith("#"):
                endpoint_pattern = endpoint_pattern.strip("#")
                if endpoint_pattern in row['url']:
                    host = urlparse(row['site_url']).hostname
                    adp_sites[endpoint_name].add(host)
            else:
                req_tld = get_tld_or_host(row['url'])
                if req_tld == endpoint_pattern:
                    host = urlparse(row['site_url']).hostname
                    adp_sites[endpoint_name].add(host)

    return adp_sites

def get_prevalence_counts(endpoint_patterns, db_path, db_path_2=None):
    # adp_sites_db = get_embedding_sites_by_regex(endpoint_regexes, db_path)
    adp_sites_db = get_embedding_sites_by_domain(endpoint_patterns, db_path)
    if db_path_2:
        adp_sites_db_2 = get_embedding_sites_by_domain(endpoint_patterns, db_path_2)
        for endpoint_name, sites in adp_sites_db_2.iteritems():
            adp_sites_db[endpoint_name] |= sites
    adp_prevalence = {endpoint_name: len(sites) for endpoint_name, sites in adp_sites_db.iteritems()}
    return adp_prevalence, adp_sites_db

## Compute prevalence using two DBs (e.g. odin and webtap)

In [ ]:
adp_prevalence, adp_sites_dict = get_prevalence_counts(ADP_PROVIDER_DOMAINS, ODIN_DB_PATH, WEBTAP_DB_PATH)
dump_as_json(adp_prevalence, "adp-third-party-lib-prevalence-odin-webtap-regex.json")

In [ ]:
for endpoint_name, site_count in adp_prevalence.iteritems():
    # print js_url, adp_sites
    print endpoint_name, site_count

## Compute prevalence using 1 DBs (census 1-million crawl DB)

In [29]:
ONE_MILLION_DB = expanduser("/mnt/10tb2/census-release-normalized/stateless/2018-11_1m_stateless/2018-11_1m_stateless_census_crawl.sqlite")

adp_prevalence, adp_sites_dict = get_prevalence_counts(ADP_PROVIDER_DOMAINS, ONE_MILLION_DB)
dump_as_json(adp_prevalence, "adp-third-party-lib-prevalence-one-million-sites.json")
for endpoint_name, site_count in adp_prevalence.iteritems():
    # print js_url, adp_sites
    print endpoint_name, site_count